Deriving z.

In [1]:
# modifying "reading in financial data" to 32 assets and right T value

import numpy as np
import pandas as pd

# You may choose to select different parameters/values
number_assets = 32
T  = 392
# Read returns
df = pd.read_csv('returns_data.txt',delim_whitespace=True)

Rraw = df.values.T

# Select the first N,T assets and scenarios, you may choose a different strategy if you would like to do so.
R = Rraw[:number_assets,:T]

# Expected return of each asset
expected_returns = np.mean(R, axis = 1)

# Covariance matrix of asset returns
covariance_matrix = np.cov(R)

/tmp/ipykernel_512/3483040339.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_512/3483040339.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv('returns_data.txt',delim_whitespace=True)


In [23]:
# import random weight function from random_weights.ipynb

def generate_random_weights(number_of_assets):
    weights = np.random.rand(number_of_assets) # Generate random numbers
    weights /= np.sum(weights) # Normalize so that the sum is 1
    return weights

# Test the function with 32 assets
n_assets = 32
random_weights = generate_random_weights(n_assets)
random_weights, sum(random_weights)


(array([5.09966343e-02, 4.53727733e-03, 1.00000515e-02, 2.29285930e-02,
        3.09233780e-02, 1.15785783e-03, 4.23591284e-02, 5.60462282e-02,
        4.28687344e-02, 2.88422553e-06, 9.16611045e-03, 2.08512285e-02,
        1.74773246e-02, 4.98874449e-02, 3.34892557e-02, 2.18674602e-02,
        1.30252699e-02, 2.88221387e-02, 3.96564751e-02, 5.59792135e-02,
        4.05992617e-02, 1.04650265e-02, 5.25108242e-02, 3.42191975e-02,
        9.83692217e-03, 5.60762032e-02, 2.96385619e-02, 5.40760816e-02,
        4.28708190e-02, 3.60760968e-02, 2.97263551e-02, 5.18619619e-02]),
 0.9999999999999999)

In [24]:
# Redefining the calculate_historical_VaR function
def calculate_historical_VaR(weights, mu_R, confidence_level=0.95):
    portfolio_returns = np.dot(mu_R, np.transpose(np.array(weights)))
    VaR = np.percentile(portfolio_returns, 100 * (1 - confidence_level))
    return -VaR

# Define a function to simulate the random generation of weights and calculation of VaR
def simulate_VaR(num_simulations, num_assets, mu_R, confidence_level=0.95):
    VaR_results = []
    weights_list = []

    for _ in range(num_simulations):
        weights = generate_random_weights(num_assets)
        VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
        VaR_results.append(VaR)
        weights_list.append(weights)
    
    return VaR_results, weights_list

In [25]:
# Example usage
num_simulations = 1000  # The number of times we want to run the simulation
num_assets = 32  # Number of assets in the portfolio

# We need the mu_R matrix to run the simulations which should be the historical returns matrix
# For the sake of the example, let's create a dummy mu_R with random values
# Normally, you would use actual historical returns data for your assets here
mu_R = np.random.randn(1000, num_assets)  # 1000 scenarios, 32 assets

# Run the simulation
VaR_results, weights_list = simulate_VaR(num_simulations, num_assets, mu_R)

# Display the first few results to ensure it's working
VaR_results[:5], weights_list[:5]

([0.33478069678017486,
  0.31380399284909316,
  0.33764572699909956,
  0.3552637321435413,
  0.3078384247834268],
 [array([0.03563346, 0.04352192, 0.02860185, 0.05366071, 0.00273621,
         0.01050274, 0.03983288, 0.00543957, 0.037661  , 0.00371156,
         0.05034161, 0.00014823, 0.00145112, 0.03447267, 0.04733192,
         0.01874436, 0.01923075, 0.04976332, 0.03784558, 0.05275445,
         0.05605801, 0.05505757, 0.03344434, 0.03201722, 0.02835223,
         0.019846  , 0.0409556 , 0.05237392, 0.03053973, 0.04146229,
         0.01626037, 0.02024677]),
  array([0.04687872, 0.032786  , 0.0125963 , 0.03430151, 0.0162518 ,
         0.0082743 , 0.02358875, 0.04415627, 0.0217607 , 0.04591394,
         0.04507806, 0.04419739, 0.04813156, 0.00641797, 0.01725279,
         0.00813625, 0.03020361, 0.04028722, 0.05129885, 0.04795799,
         0.02350275, 0.01127891, 0.00113751, 0.04436431, 0.04858859,
         0.02265057, 0.02337002, 0.0431164 , 0.0352238 , 0.04258584,
         0.04689514, 0.

In [12]:
# let's now actually calculate z


# Parameters
number_of_assets = 32
number_of_portfolios = 1000  # The number of random portfolios you want to generate
confidence_level = 0.95

# Assuming mu_R is matrix of historical returns for the assets
# Replace this with our actual historical returns data
mu_R = np.random.randn(1000, number_of_assets)  # This should be the historical returns data for your assets

# Generate the random weights for the portfolios
random_weights = generate_random_weights(number_of_assets, number_of_portfolios)

# Calculate the VaR for each portfolio
VaRs = []
for weights in random_weights:
    VaR = calculate_historical_VaR(weights, mu_R, confidence_level)
    VaRs.append(VaR)

# Find the z value for your constraint equations
# This could be the mean VaR, median VaR, or some other statistic that you determine is appropriate for your needs
mean_VaR = np.mean(VaRs)
median_VaR = np.median(VaRs)
z = mean_VaR  # or median_VaR, depending on your specific requirements

print(f"The mean VaR at a {confidence_level*100}% confidence level is: {mean_VaR}")
print(f"The median VaR at a {confidence_level*100}% confidence level is: {median_VaR}")

TypeError: generate_random_weights() takes 1 positional argument but 2 were given